# 1. Crawler

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [ ]:
import argparse
import os
import re
import requests
import hashlib


def wget(url, filename):
    # allow redirects - in case file is relocated
    resp = requests.get(url, allow_redirects=True)
    # this can also be 2xx, but for simplicity now we stick to 200
    # you can also check for `resp.ok`
    if resp.status_code != 200:
        print(resp.status_code, resp.reason, 'for', url)
        return

    # just to be cool and print something
    print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
    print()

    # try to extract filename from url
    if filename is None:
        # start with http*, ends if ? or # appears (or none of)
        m = re.search("^http.*/([^/\?#]*)[\?#]?", url)
        filename = m.group(1)
        if not filename:
            filename = hashlib.sha256(url.encode()).hexdigest()
            # raise NameError(f"Filename neither given, nor found for {url}")

    # what will you do in case 2 websites store file with the same name?
    if os.path.exists(filename):
        filename = hashlib.sha256(filename.encode()).hexdigest()
        # raise OSError(f"File {filename} already exists")

    with open(filename, 'wb') as f:
        f.write(resp.content)
        print(f"File saved as {filename}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='download file.')
    parser.add_argument("-O", type=str, default=None, dest='filename',
                        help="output file name. Default -- taken from resource")
    parser.add_argument("url", type=str, default=None, help="Provide URL here")
    args = parser.parse_args()
    wget(args.url, args.filename)


### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [10] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [1]:
from os.path import exists
import requests
from urllib.parse import quote
import hashlib


class Document:

    def __init__(self, url):
        self.url = url
        self.content = None

    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()

    def download(self):
        #TODO download self.url content, store it in self.content and return True in case of success
        response = requests.get(self.url, allow_redirects=True)
        if response.status_code != requests.codes.ok:
            return False
        if response.content is None:
            return False
        self.content = response.content
        return True

    def persist(self):
        #TODO write document content to hard drive
        filename = hashlib.sha256(self.url.encode()).hexdigest()
        with open(filename, 'wb') as f:
            f.write(self.content)

    def load(self):
        #TODO load content from hard drive, store it in self.content and return True in case of success
        filename = hashlib.sha256(self.url.encode()).hexdigest()
        if not exists(filename):
            return False
        with open(filename, 'rb') as f:
            self.content = f.read()
        return True

### 1.1.1. Tests ###

In [3]:
doc = Document('https://github.com/YusufRoshdy/information-retrieval/raw/main/datasets/facts.txt')

doc.get()
assert doc.content, "Document download failed"
assert "You breathe on average about 8,409,600 times a year" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "You breathe on average about 8,409,600 times a year" in str(doc.content), "Document load from disk error"

b'1. If you somehow found a way to extract all of the gold from the bubbling core of our lovely little planet, you would be able to cover all of the land in a layer of gold up to your knees.\n2. McDonalds calls frequent buyers of their food "heavy users."\n3. The average person spends 6 months of their lifetime waiting on a red light to turn green.\n4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.\n5. You burn more calories sleeping than you do watching television.\n6. There are more lifeforms living on your skin than there are people on the planet.\n7. Southern sea otters have flaps of skin under their forelegs that act as pockets. When diving, they use these pouches to store rocks and food.\n8. In 1386 a pig in France was executed by public hanging for the murder of a child.\n9. One in every five adults believe that aliens are hiding in our planet disguised as humans.\n10. If you believe that you\'re truly one in a million, there are still 

## 1.2. [10] Parse HTML
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
1. `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
2. `self.images` list of images met in a document. Again, links can be relative to current page.
3. `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [16]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse


class HtmlDocument(Document):

    def parse(self):
        #TODO extract plain text, images and links from the document
        self.get()
        soup = BeautifulSoup(self.content, 'html.parser')

        links = soup.find_all('a', href=True)
        self.anchors = [(a.text.strip(), urllib.parse.urljoin(self.url, a['href'].strip())) for a in links]

        images = soup.find_all('img')
        self.images = [urllib.parse.urljoin(self.url, img['src'].strip()) for img in images]

        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True

        visible_texts = filter(tag_visible, soup.findAll(string=True))
        self.text = " ".join(t.strip() for t in visible_texts)


### 1.2.1. Tests

In [17]:
doc = HtmlDocument("https://innopolis.university/en/")
doc.get()
doc.parse()

assert "Education, research and development" in doc.text, "Error parsing text"
assert "https://innopolis.university/upload/resize_cache/iblock/e5c/510_340_2/deadline_extended.jpg" in doc.images, "Error parsing images"
assert any(p[1] == "https://innopolis.university/en/faculty/" for p in doc.anchors), "Error parsing links"

/tmp/ipykernel_10072/1781702317.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  visible_texts = filter(tag_visible, soup.findAll(text=True))


## 1.3. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose).

**Criteria to succeed in the task**:
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained from inside `<body>` tag only.

In [30]:
from collections import Counter


class HtmlDocumentTextData:

    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()

    def get_sentences(self):
        #TODO implement sentence parser
        result = []
        return result

    def get_word_stats(self):
        #TODO return Counter object of the document, containing mapping {`word` -> count_in_doc}
        return Counter()

### 1.3.1. Tests ###

In [ ]:
doc = HtmlDocumentTextData("https://innopolis.university/en")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'innopolis'], 'innopolis should be among most common'

## 1.4 [10] Account the caching policy
Sometimes remote documents (especially when we speak about static content like `js` or `gif`) can swear that they will not change for some time. This is done by setting [Cache-Control response header](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Cache-Control).

In [ ]:
import requests

requests.get('https://polyfill.io/v3/polyfill.min.js').headers['Cache-Control']

Please study the documentation and implement a descendant to a Document class, which will refresh the document in case of expired cache even if the file is already on the hard drive.

In [38]:
class CachedDocument(Document):
    # TODO your code here
    pass

### 1.4.1. Tests ###

Add logging to your code and show that your code behaves differently for documents with different caching policy.

In [ ]:
import time

doc = CachedDocument('https://polyfill.io/v3/polyfill.min.js')
doc.get()
time.sleep(2)
doc.get()
time.sleep(2)
doc.get()

doc = CachedDocument('https://yandex.ru/')
doc.get()
time.sleep(2)
doc.get()
time.sleep(2)
doc.get()

## 1.5. [10] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [34]:
from queue import Queue


class Crawler:

    def crawl_generator(self, source, depth=1):
        #TODO return real crawling results. Don't forget to process failures,
        # exceptions, 3**, 4** codes
        for i in range(3):
            yield HtmlDocumentTextData(source)

### 1.5.1. Tests ###

In [ ]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")

print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'